Mục tiêu: 
    Phát hiện lỗi tiềm ẩn, nhiễu, giá trị thiếu
    Quan sát các xu hướng
    Quan sát các đặc trưng đóng vai trò quan trọng
    Tìm ra các vi phạm hoặc các giả thiết thống kê có thể có từ các quan sát
    Giảm chiều, tăng cường


Các bước phân tích và trực quan hóa dữ liệu 



In [1]:
print("Hello, Jupyter!")


Hello, Jupyter!
